In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Importing dataset

In [ ]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_3f9688f9615b4905ac51c6a765daa457 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='t7WnXYT6ucf53aWRT1Z_gSFB29ciLfPgYzFvW_3K5ijf',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_3f9688f9615b4905ac51c6a765daa457.get_object(Bucket='lifeexpectancydata-donotdelete-pr-2qg4cpdsrzfzus',Key='Life Expectancy Data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head()

# Data Visualisation

In [ ]:
plt.figure(figsize = (14, 10))
sns.heatmap(data.corr(), annot = True)

In [ ]:
print('\n Data shape: ',data.shape)
print('\n Columns in data set: ',data.columns)

# Data Preprocessing

In [ ]:
print('\n',data.info())

In [ ]:
print('\n',data.corr())

In [ ]:
print('\n',data.isnull().sum(axis=0))

In [ ]:
print('\n',data.isnull().sum(axis=0))

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
data['Status'] = lb.fit_transform(data['Status'])

In [ ]:
data = data.drop('Country', axis=1)
data.fillna(value=data.mean(),inplace=True)
data_shape = data.shape
data_shape

In [ ]:
X = data.drop(['Life expectancy '], axis=1).values
y = data['Life expectancy '].values

# Implementing linear regression model

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state=75)
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train,y_train)

In [ ]:
y_prediction = lr.predict(X_test)
y_prediction

In [ ]:
from sklearn.metrics import mean_squared_error, accuracy_score, mean_absolute_error
print('\naccuracy(R^2): {}%'.format(lr.score(X_test, y_test)*100))
print('\nmae: {}'.format(mean_absolute_error(y_test, y_prediction)))
print('\nrmse: {}'.format(np.sqrt(mean_squared_error(y_test, y_prediction))))

In [ ]:
#LR_scatter_graph
sns.scatterplot(x=y_test,y=y_prediction)
plt.show()

In [ ]:


demo_pred=lr.predict([[2015,0,263,62,0.01,71.27962362,65,1154,19.1,83,6,8.16,65,0.1,584.25921,33736494,17.2,17.3,0.479,10.1]])
demo_pred

#  Done training and testing of model! 
# Deployment
## Run the following code, only if you are working with IBM watson studio

In [ ]:
!pip install watson-machine-learning-client


In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient


In [ ]:
#this all can get from IBM cloud machine learning service credentials
wml_credentials={
  "apikey": "--" ,
   "instance_id": "--",
  "url": "--"
   }

In [ ]:
client=WatsonMachineLearningAPIClient(wml_credentials)


In [ ]:
 model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "--", 
               client.repository.ModelMetaNames.AUTHOR_EMAIL: "--", 
                client.repository.ModelMetaNames.NAME: "--"}

In [ ]:
model_artifact =client.repository.store_model(lr, meta_props=model_props)

In [ ]:
published_model_uid = client.repository.get_model_uid(model_artifact)

published_model_uid

In [ ]:
deployment = client.deployments.create(published_model_uid, name="LifeExpectancyPrediction")

In [ ]:
scoring_endpoint = client.deployments.get_scoring_url(deployment)

In [ ]:
scoring_endpoint